# hunyuanOCR测试

In [15]:
# VS Code中不能加载谷歌网盘

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
!pwd
!ls -la

/content
total 36
drwxr-xr-x 1 root root  4096 Jan 30 17:31 .
drwxr-xr-x 1 root root  4096 Jan 30 17:05 ..
drwxr-xr-x 4 root root  4096 Dec  9 14:41 .config
drwx------ 6 root root  4096 Jan 30 17:20 drive
-rw-r--r-- 1 root root 11463 Jan 30 17:42 hunyuan_ocr_results.json
-rw-r--r-- 1 root root  1499 Jan 30 17:31 hunyuan_ocr.txt
drwxr-xr-x 1 root root  4096 Dec  9 14:42 sample_data


运行上述代码后，会弹出一个认证链接。点击该链接，选择您的 Google 账号并授权，然后将授权码复制回 Colab 的输入框中，按回车键即可。

挂载成功后，您就可以通过 `/content/drive/MyDrive/` 路径访问您的 Google Drive 中的文件了，例如，如果您的文件在 Google Drive 的根目录下的 `my_data` 文件夹中，那么在 Colab 中它的路径就是 `/content/drive/MyDrive/my_data/`。

In [17]:
import os

# Check if Google Drive is mounted
if os.path.exists('/content/drive/MyDrive'):
    print('Google Drive is already mounted.')
    # List contents of your MyDrive to confirm
    print('Contents of MyDrive:')
    print(os.listdir('/content/drive/MyDrive'))
else:
    print('Google Drive does not appear to be mounted.')


Google Drive is already mounted.
Contents of MyDrive:
['Colab Notebooks', 'Google Photos', 'Google Earth', 'Introduction_to_the_Wênchow_Dialect.pdf', 'party', '.ipynb_checkpoints', 'DeepSeekOCR', 'Documents', 'shanghai_gazetteer', 'shanghai_gazetteer_cleaned', 'shanghai_gazetteer_finetuned_model', '新汉文典', 'Introduction_to_the_Wênchow_Dialect.gdoc', '.vscode']


如果上述代码显示 Google Drive 已经挂载，并且您能看到 `MyDrive` 的内容，那么说明一切正常，您无需担心没有弹出链接。

In [18]:
!pip install git+https://github.com/huggingface/transformers@82a06db03535c49aa987719ed0746a76093b1ec4


  Cloning https://github.com/huggingface/transformers (to revision 82a06db03535c49aa987719ed0746a76093b1ec4) to /tmp/pip-req-build-sqfogh6m
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-sqfogh6m
  Running command git rev-parse -q --verify 'sha^82a06db03535c49aa987719ed0746a76093b1ec4'
  Running command git fetch -q https://github.com/huggingface/transformers 82a06db03535c49aa987719ed0746a76093b1ec4
  Running command git checkout -q 82a06db03535c49aa987719ed0746a76093b1ec4
  Resolved https://github.com/huggingface/transformers to commit 82a06db03535c49aa987719ed0746a76093b1ec4
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [19]:
from transformers import AutoProcessor
from transformers import HunYuanVLForConditionalGeneration
from PIL import Image
import torch

def clean_repeated_substrings(text):
    """Clean repeated substrings in text"""
    n = len(text)
    if n<8000:
        return text
    for length in range(2, n // 10 + 1):
        candidate = text[-length:]
        count = 0
        i = n - length

        while i >= 0 and text[i:i + length] == candidate:
            count += 1
            i -= length

        if count >= 10:
            return text[:n - length * (count - 1)]

    return text

model_name_or_path = "tencent/HunyuanOCR"
processor = AutoProcessor.from_pretrained(model_name_or_path, use_fast=False)
img_path = "/content/drive/MyDrive/DeepSeekOCR/page_135.png"
image_inputs = Image.open(img_path)
messages1 = [
    {"role": "system", "content": ""},
    {
        "role": "user",
        "content": [
            {"type": "image", "image": img_path},
            {"type": "text", "text": (
                "检测并识别图片中的文字，将文本坐标格式化输出。"
            )},
        ],
    }
]
messages = [messages1]
texts = [
    processor.apply_chat_template(msg, tokenize=False, add_generation_prompt=True)
    for msg in messages
]
inputs = processor(
    text=texts,
    images=image_inputs,
    padding=True,
    return_tensors="pt",
)
model = HunYuanVLForConditionalGeneration.from_pretrained(
    model_name_or_path,
    attn_implementation="eager",
    dtype=torch.bfloat16,
    device_map="auto"
)
with torch.no_grad():
    device = next(model.parameters()).device
    inputs = inputs.to(device)
    generated_ids = model.generate(**inputs, max_new_tokens=16384, do_sample=False)
if "input_ids" in inputs:
    input_ids = inputs.input_ids
else:
    print("inputs: # fallback", inputs)
    input_ids = inputs.inputs
generated_ids_trimmed = [
    out_ids[len(in_ids):] for in_ids, out_ids in zip(input_ids, generated_ids)
]
output_texts = clean_repeated_substrings(processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
))
print(output_texts)



Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

['二都在大海中崖岸之邉出真珠故曰珠崖儋(863,175),(900,860)耳者種大耳渠帥目謂王者耳尤鍐下肩三寸(825,176),(864,861)張晏曰異物志二郡在海中東西千里南北五(778,176),(817,859)百里珠崖言珠若崖矣儋耳之云鏤其頰皮上(739,178),(778,859)連耳匡分爲數支状似鷄腸累耳下𡸁仰古曰(694,176),(736,851)儋音丁甘反字本作瞻臣瓉曰茂陵書珠崖郡(656,176),(696,861)去長安七千三百一十四里儋曰茂陵書珠崖(616,173),(658,861)耳去長安七千三百六十八里(578,173),(615,630)領縣五(578,630),(628,761)據茂陵(605,761),(638,861)書瑇瑁(564,763),(600,861)紫貝苟中至來九龍#賈捐之傳又曰山南縣(532,173),(570,861)反又曰合十六縣反則縣不止五盖(493,173),(530,861)暉都亦(461,171),(494,300)縣名(422,171),(454,250)督扵交州剌史昭帝始元五年省儋耳(404,300),(460,861)入珠厓(391,171),(430,300)賈捐之傳自初爲郡至昭帝始元元年(378,303),(428,861)二十餘年間凡六反叛至其五年罷儋(341,305),(393,861)耳郡并属珠厓○外紀(320,171),(358,500)云用封爵詔言者省(285,171),(322,477)元帝初元三年罷棄其(288,500),(338,861)郡(234,171),(274,221)用賈捐之䟽○地理志自初爲郡縣特卒中(246,224),(290,856)國人多侵陵之故率數歲一反元帝特遂罷(206,225),(254,856)棄之後南蠻傳中國貪其珍賭新相侵侮故率(165,171),(216,859)數歲一反至元帝罷之珠厓立郡凡六十五歲(128,173),(178,859)']


# Task
The current task is to complete the batch OCR processing for all `.png` image files in the `/content/drive/MyDrive/新汉文典/data/black_white_revert/` directory, save the accumulated OCR results to a file named `ocr_results.txt` in the same directory, and then verify the content of the saved file.

## Save OCR Results to File

### Subtask:
Iterate through the `ocr_results` list and write each image's filename and its corresponding OCR text to a file named `ocr_results.txt` within the `image_dir` (the same directory where the images are located). Each entry will be clearly separated.


**Reasoning**:
I need to write the `ocr_results` to a text file, I will iterate through the list and write each filename and its OCR text to `ocr_results.txt` in the `image_dir`.



In [20]:
!pwd
!ls -la
output_path = "hunyuan_ocr.txt"  # 改成你要的路径
with open(output_path, "w", encoding="utf-8") as f:
    f.write("\n".join(output_texts))

print(f"Saved to: {output_path}")


/content
total 36
drwxr-xr-x 1 root root  4096 Jan 30 17:31 .
drwxr-xr-x 1 root root  4096 Jan 30 17:05 ..
drwxr-xr-x 4 root root  4096 Dec  9 14:41 .config
drwx------ 6 root root  4096 Jan 30 17:20 drive
-rw-r--r-- 1 root root 11463 Jan 30 17:42 hunyuan_ocr_results.json
-rw-r--r-- 1 root root  1499 Jan 30 17:31 hunyuan_ocr.txt
drwxr-xr-x 1 root root  4096 Dec  9 14:42 sample_data
Saved to: hunyuan_ocr.txt


## Verify Saved File

### Subtask:
Confirm that the `ocr_results.txt` file has been successfully created in the `image_dir` and contains the expected OCR results by reading a portion of it.


**Reasoning**:
To verify the content of the `ocr_results.txt` file, I will open it, read the first 500 characters, and print them to the console.



In [21]:
with open(output_path, "r", encoding="utf-8") as f:
    first_500_chars = f.read(500)

print("\n--- Verifying Saved OCR Results (first 500 characters) ---")
print(first_500_chars)
print("-----------------------------------------------------------")


--- Verifying Saved OCR Results (first 500 characters) ---
二都在大海中崖岸之邉出真珠故曰珠崖儋(863,175),(900,860)耳者種大耳渠帥目謂王者耳尤鍐下肩三寸(825,176),(864,861)張晏曰異物志二郡在海中東西千里南北五(778,176),(817,859)百里珠崖言珠若崖矣儋耳之云鏤其頰皮上(739,178),(778,859)連耳匡分爲數支状似鷄腸累耳下𡸁仰古曰(694,176),(736,851)儋音丁甘反字本作瞻臣瓉曰茂陵書珠崖郡(656,176),(696,861)去長安七千三百一十四里儋曰茂陵書珠崖(616,173),(658,861)耳去長安七千三百六十八里(578,173),(615,630)領縣五(578,630),(628,761)據茂陵(605,761),(638,861)書瑇瑁(564,763),(600,861)紫貝苟中至來九龍#賈捐之傳又曰山南縣(532,173),(570,861)反又曰合十六縣反則縣不止五盖(493,173),(530,861)暉都亦(461,171),(494,300)縣名(422,171),(454,250)督扵交州剌史昭帝始元五年省儋耳(404,300),(460,8
-----------------------------------------------------------


## List and Sort Image Files

### Subtask:
Identify all .png image files in the specified directory and sort them numerically based on their page number, from smallest to largest page number. This ensures processing in the correct book order.


**Reasoning**:
To identify and numerically sort the .png image files, I will define the image directory, list all .png files using `os.listdir` and `endswith('.png')`, and then sort them by extracting the numerical page number from each filename using a regular expression to ensure correct book order.



In [22]:

import os
import re

image_dir = '/content/drive/MyDrive/DeepSeekOCR'

def extract_page_number(filename):
    match = re.search(r'page_(\d+)\.png', filename)
    if match:
        return int(match.group(1))
    return -1  # Return -1 for files that don't match the pattern, placing them at the beginning if not handled otherwise

# Get all .png files and sort them numerically by page number
image_files = sorted(
    [f for f in os.listdir(image_dir) if f.endswith('.png')],
    key=extract_page_number
)

print(f"Found {len(image_files)} PNG image files in {image_dir}:")
for img_file in image_files:
    print(img_file)

Found 13 PNG image files in /content/drive/MyDrive/DeepSeekOCR:
page_15.png
page_16.png
page_17.png
page_18.png
page_19.png
page_24.png
page_25.png
page_135.png
page_136.png
page_150.png
page_168.png
page_169.png
page_177.png


## Perform Batch OCR

### Subtask:
Iterate through each identified and sorted image file. For each image, apply the existing OCR processing logic using the `HunyuanVLForConditionalGeneration` model and the `AutoProcessor`. Collect the OCR output for each image.


**Reasoning**:
I need to iterate through the sorted `image_files` list, perform OCR on each image using the `HunyuanVLForConditionalGeneration` model and `AutoProcessor`, and collect the results in the `ocr_results` list as specified in the subtask instructions.



In [23]:
from PIL import Image
import torch

ocr_results = []

for img_filename in image_files:
    full_img_path = os.path.join(image_dir, img_filename)
    image_inputs = Image.open(full_img_path)

    # Prepare messages for the model
    messages_for_ocr = [
        {"role": "system", "content": ""},
        {
            "role": "user",
            "content": [
                {"type": "image", "image": full_img_path},
                {"type": "text", "text": (
                    "检测并识别图片中的文字，将文本坐标格式化输出。"
                )},
            ],
        }
    ]

    # Apply chat template and tokenize
    texts = [processor.apply_chat_template(messages_for_ocr, tokenize=False, add_generation_prompt=True)]
    inputs = processor(
        text=texts,
        images=image_inputs,
        padding=True,
        return_tensors="pt",
    )

    # Move inputs to device and generate output
    with torch.no_grad():
        device = next(model.parameters()).device
        inputs = inputs.to(device)
        generated_ids = model.generate(**inputs, max_new_tokens=16384, do_sample=False)

    # Trim generated IDs and decode
    if "input_ids" in inputs:
        input_ids = inputs.input_ids
    else:
        input_ids = inputs.inputs # fallback

    generated_ids_trimmed = [
        out_ids[len(in_ids):] for in_ids, out_ids in zip(input_ids, generated_ids)
    ]
    output_text = clean_repeated_substrings(processor.batch_decode(
        generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
    )[0]) # Get the first (and only) item from the batch_decode list

    ocr_results.append({"filename": img_filename, "ocr_text": output_text})
    print(f"Processed {img_filename}")

print("Batch OCR processing complete.")

Processed page_15.png
Processed page_16.png
Processed page_17.png
Processed page_18.png
Processed page_19.png
Processed page_24.png
Processed page_25.png
Processed page_135.png
Processed page_136.png
Processed page_150.png
Processed page_168.png
Processed page_169.png
Processed page_177.png
Batch OCR processing complete.


以上只是把结果放进 ocr_results 列表里，并没有写入文件。要保存，最后加一段写文件即可，例如保存成 JSON：

In [24]:
import json
!pwd
!ls -la

output_path = "hunyuan_ocr_results.json"
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(ocr_results, f, ensure_ascii=False, indent=2)

print(f"Saved to: {output_path}")


/content
total 36
drwxr-xr-x 1 root root  4096 Jan 30 17:31 .
drwxr-xr-x 1 root root  4096 Jan 30 17:05 ..
drwxr-xr-x 4 root root  4096 Dec  9 14:41 .config
drwx------ 6 root root  4096 Jan 30 17:20 drive
-rw-r--r-- 1 root root 11463 Jan 30 17:42 hunyuan_ocr_results.json
-rw-r--r-- 1 root root  1499 Jan 30 17:48 hunyuan_ocr.txt
drwxr-xr-x 1 root root  4096 Dec  9 14:42 sample_data
Saved to: hunyuan_ocr_results.json


## Verify Saved File

### Subtask:
Confirm that the `hunyuan_ocr_results.json` file has been successfully created in the `image_dir` and contains the expected OCR results by reading a portion of it.

**Reasoning**:
To verify the content of the `ocr_results.txt` file, I will open it, read the first 500 characters, and print them to the console, as indicated in the previous reasoning block.



In [25]:
with open(output_file_path, "r", encoding="utf-8") as f:
    first_500_chars = f.read(500)

print("\n--- Verifying Saved OCR Results (first 500 characters) ---")
print(first_500_chars)
print("-----------------------------------------------------------")


--- Verifying Saved OCR Results (first 500 characters) ---
Filename: page_15.png
OCR Text: 代及我(794,128),(870,271)祖宗致治太平事跡幡然喜曰誠䏻盡職(702,129),(781,847)了𬼘志中事目報國其庶幾乎扵是訪延(619,129),(690,845)郡中士夫若干人相與稽考典籍自漢唐(526,129),(602,849)宋以來歸宿扵(448,129),(517,395)皇朝要重脩郡志以新海外一代耳目縁(363,129),(444,849)唐宋古志無徵而國朝𦾔志亦多蠧落乃(278,129),(357,847)廣搜愽采國初謫宦諸老儒之家存有殘(192,129),(270,847)篇遺集而濫及不祿自昧寡陋著爲瓊䑓(104,129),(185,849)
--------------------------------------------------

Filename: page_16.png
OCR Text: 外紀一書亦得側於諸家遺集以補郡志(807,132),(876,846)之缺略自郡縣建置沿革至扵山川形勝(727,133),(801,847)軍衛城池學校人物户口貢賦等事與凡(648,
-----------------------------------------------------------
